In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

In [2]:
pd.set_option('display.max_colwidth', 1200)

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

## Study data train

In [79]:
df_pred = pd.read_csv("data/data_predict/df_train_10_pred_intersection.csv")

In [80]:
df_pred.head()

,Unnamed: 0,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
0,0,GPT,"de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina",glutamic pyruvic transaminar,1,"[CLS] glutamic pyruvic transaminar [SEP] de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina [SEP]",0.0,0.0,0.0
1,1,GPT,"de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina",glutamate pyruvatar transaminar,0,"[CLS] glutamate pyruvatar transaminar [SEP] de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina [SEP]",0.0,0.0,0.0
2,2,C,"A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal",centimetro,1,"[CLS] centimetro [SEP] A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal [SEP]",0.0,0.0,0.0
3,3,C,"A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal",centigrado,0,"[CLS] centigrado [SEP] A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal [SEP]",0.0,0.0,0.0
4,4,C,"A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal",cysteine,0,"[CLS] cysteine [SEP] A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal [SEP]",0.0,0.0,0.0


In [36]:
df_pred.shape

(2232, 9)

Veamos cuantos acrónimos son ambiguos

In [37]:
amb = df_pred.groupby('short_form')['long_form'].nunique().reset_index().sort_values('long_form', ascending = False)
print(amb.shape)
amb[amb['long_form'] == 1].shape

(54, 2)


(0, 2)

Filtramos para aquellas no ha acertado

In [81]:
wrong = df_pred[((df_pred['label'] == 0) & (df_pred['Prediction_train1'] == 1.0)) | (df_pred['label'] == 1) & (df_pred['Prediction_train1'] == 0.0)]

In [82]:
wrong.shape

(109, 9)

In [83]:
del wrong['Unnamed: 0']

In [84]:
wrong.sort_values(['short_form'])

,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
194,ALT,"predominio colestásico ( fosfatasa alcalina y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) , sin insuficiencia hepática .",alanine transferar,0,"[CLS] alanine transferar [SEP] predominio colestásico ( fosfatasa alcalina y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) , sin insuficiencia hepática . [SEP]",1.0,0.0,0.0
191,ALT,perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH,alanine transferar,0,[CLS] alanine transferar [SEP] perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH [SEP],1.0,0.0,0.0
190,ALT,perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH,alanine transaminar,1,[CLS] alanine transaminar [SEP] perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH [SEP],0.0,0.0,0.0
188,AST,"( fosfatasa alcalina y GGT más de 10 veces el valor normal y ALT y < start > AST < end > menos de 3 veces el valor normal ) , sin insuficiencia hepática . A las",aspartate and alanine aminotransferar,0,"[CLS] aspartate and alanine aminotransferar [SEP] ( fosfatasa alcalina y GGT más de 10 veces el valor normal y ALT y < start > AST < end > menos de 3 veces el valor normal ) , sin insuficiencia hepática . A las [SEP]",1.0,0.0,0.0
184,AST,presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( ALT y < start > AST < end > mayores de 20 veces el valor normal con importante aumento de la LDH ) y,aspartato aminotransferasar,1,[CLS] aspartato aminotransferasar [SEP] presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( ALT y < start > AST < end > mayores de 20 veces el valor normal con importante aumento de la LDH ) y [SEP],0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
64,mm,"nueva dilatación del píloro , en este caso forzada , con un balón de 35 < start > mm < end > ( Rigiflex® ; Microvasive , Boston Scientific , EE . UU . ) , previo",milimetro,1,"[CLS] milimetro [SEP] nueva dilatación del píloro , en este caso forzada , con un balón de 35 < start > mm < end > ( Rigiflex® ; Microvasive , Boston Scientific , EE . UU . ) , previo [SEP]",0.0,0.0,0.0
66,mm,"píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un",milimetro,1,"[CLS] milimetro [SEP] píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un [SEP]",0.0,0.0,0.0
67,mm,"píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un",milimetro mercurio,0,"[CLS] milimetro mercurio [SEP] píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un [SEP]",1.0,0.0,0.0
68,mm,"Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un gastroscopio de 10 < start > mm < end > de forma fácil por dicho canal . Sin embargo , después de unos días ,",milime

In [85]:
wrong.groupby('short_form').size().reset_index().sort_values(0, ascending = False)

,short_form,0
12,cm,42
2,C,12
6,H2O,11
8,PCR,7
15,mm,7
13,h,5
7,L,5
1,AST,4
14,kg,3
0,ALT,3


Estudiemos cada acrónimo que da fallo

In [51]:
wrong[wrong['short_form'] == 'ALT']

,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
190,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transaminar,1,[CLS] alanine transaminar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],0.0,0.0,0.0
191,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transferar,0,[CLS] alanine transferar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],1.0,0.0,0.0
194,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transferar,0,[CLS] alanine transferar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0


In [50]:
df_pred[(df_pred['short_form'] == 'ALT')]['long_form'].unique()

array(['alanine transaminar', 'alanine transferar',
       'alanina aminotransferaso'], dtype=object)

In [49]:
df_pred[(df_pred['short_form'] == 'ALT') & (df_pred['label'] == 1)]

,Unnamed: 0,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
190,190,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transaminar,1,[CLS] alanine transaminar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],0.0,0.0,0.0
193,193,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transaminar,1,[CLS] alanine transaminar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0
459,459,ALT,"mg/dl , bilirrubina 1,1 mg/dl , AST 141 UI/l , < start > ALT < end > 38 UI/l , Fosfatasa alcalina 245 UI/l , LDH 1650",alanine transferar,1,"[CLS] alanine transferar [SEP] mg/dl , bilirrubina 1,1 mg/dl , AST 141 UI/l , < start > ALT < end > 38 UI/l , Fosfatasa alcalina 245 UI/l , LDH 1650 [SEP]",NaN,NaN,NaN
717,717,ALT,valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ),alanine transferar,1,[CLS] alanine transferar [SEP] valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ) [SEP],NaN,NaN,NaN
720,720,ALT,importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ),alanine transferar,1,[CLS] alanine transferar [SEP] importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ) [SEP],NaN,NaN,NaN
734,734,ALT,valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ),alanina aminotransferaso,1,[CLS] alanina aminotransferaso [SEP] valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ) [SEP],NaN,NaN,NaN
737,737,ALT,importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ),alanina aminotransferaso,1,[CLS] alanina aminotransferaso [SEP] importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ) [SEP],NaN,NaN,NaN
1058,1058,ALT,"fracción directa de 5,7 mg/dl , AST 312 U/l , < start > ALT < end > 106 U/l , GGT 416 U/l . Con el diagnóstico",alanine transferar,1,"[CLS] alanine transferar [SEP] fracción directa de 5,7 mg/dl , AST 312 U/l , < start > ALT < end > 106 U/l , GGT 416 U/l . Con el diagnóstico [SEP]",NaN,NaN,NaN
1076,1076,ALT,"Globulinas 3.2 , Fosfatasa alcalina 255 , GGT 22 , < start > ALT < end > 49 , AST 65 , Glicemia 148 y Creatinina 0.55",alanine transferar,1,"[CLS] alanine transferar [SEP] Globulinas 3.2 , Fosfatasa alcalina 255 , GGT 22 , < start > ALT < end > 49 , AST 65 , Glicemia 148 y Creatinina 0.55 [SEP]",NaN,NaN,NaN
1087,1087,ALT,91mg/dl ; bilirrubina total 21mg/dl ; AST 164 U/l ; < start > ALT < end > 103 U/l ; GGT 152 UI/dl . Marcadores virales hepáticos,alanina aminotransferaso,1,[CLS] alanina aminotransferaso [SEP] 91mg/dl ; bilirrubina total 21mg/dl ; AST 164 U/l ; < start > ALT < end > 103 U/l ; GGT 152 UI/dl . Marcadores virales hepáticos [SEP],NaN,NaN,NaN


#### Stuydy LF with less rows

In [69]:
train = pd.read_csv("data/data_train/train_data_beto_10_amb_lfnorm.csv")

In [70]:
train.head()

,Unnamed: 0,short_form,context,long_form,label
0,0,GPT,"bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran",glutamic pyruvic transaminar,1
1,1,GPT,"bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran",glutamate pyruvatar transaminar,0
2,2,C,"la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada ,",centimetro,1
3,3,C,"la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada ,",centigrado,0
4,4,C,"la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada ,",cysteine,0


In [72]:
train.groupby('long_form').size().reset_index().sort_values(0)

,long_form,0
43,cumulo diferenciacion 30,2
112,tomografia computarizado,2
20,beta,2
19,basofilo,2
113,tubular,2
...,...,...
81,microgramo,110
83,milimetro,114
84,milimetro mercurio,118
25,centimetro agua,227


## Study data test

In [52]:
test = pd.read_csv("data/data_predict/df_test_10_pred_amb_lfnorm.csv")

In [53]:
test.head()

,Unnamed: 0,Unnamed: 0.1,short_form,context,long_form,label,sentences,Prediction
0,0,0,CA,IgG 1840 UI/ml ( 751-1.560 UI/ml ) . El marcador < start > CA < end > 19-9 era de 82 UI/ml . Los autoanticuerpos ( ANA,carbohydrate antiger,1,[CLS] carbohydrate antiger [SEP] IgG 1840 UI/ml ( 751-1.560 UI/ml ) . El marcador < start > CA < end > 19-9 era de 82 UI/ml . Los autoanticuerpos ( ANA [SEP],0
1,1,1,CA,IgG 1840 UI/ml ( 751-1.560 UI/ml ) . El marcador < start > CA < end > 19-9 era de 82 UI/ml . Los autoanticuerpos ( ANA,california,0,[CLS] california [SEP] IgG 1840 UI/ml ( 751-1.560 UI/ml ) . El marcador < start > CA < end > 19-9 era de 82 UI/ml . Los autoanticuerpos ( ANA [SEP],0
2,2,2,CA,IgG 1840 UI/ml ( 751-1.560 UI/ml ) . El marcador < start > CA < end > 19-9 era de 82 UI/ml . Los autoanticuerpos ( ANA,cancer antiger,0,[CLS] cancer antiger [SEP] IgG 1840 UI/ml ( 751-1.560 UI/ml ) . El marcador < start > CA < end > 19-9 era de 82 UI/ml . Los autoanticuerpos ( ANA [SEP],0
3,3,3,FA,"UI/ml , GPT 371 UI/ml , GGT 632 UI/ml , < start > FA < end > 787 UI/ml , LDH 394 UI/ml . Amilasa y lipasa",fosfatasa alcalino,1,"[CLS] fosfatasa alcalino [SEP] UI/ml , GPT 371 UI/ml , GGT 632 UI/ml , < start > FA < end > 787 UI/ml , LDH 394 UI/ml . Amilasa y lipasa [SEP]",0
4,4,4,FA,"UI/ml , GPT 371 UI/ml , GGT 632 UI/ml , < start > FA < end > 787 UI/ml , LDH 394 UI/ml . Amilasa y lipasa",faceta articular,0,"[CLS] faceta articular [SEP] UI/ml , GPT 371 UI/ml , GGT 632 UI/ml , < start > FA < end > 787 UI/ml , LDH 394 UI/ml . Amilasa y lipasa [SEP]",0


In [54]:
del test['Unnamed: 0']
del test['Unnamed: 0.1']

In [55]:
test.shape

(1696, 6)

Filtramos para aquellas no ha acertado

In [56]:
wrong_test = test[((test['label'] == 0) & (test['Prediction'] == 1.0)) | (test['label'] == 1) & (test['Prediction'] == 0.0)]

In [57]:
wrong_test.shape

(713, 6)

In [59]:
wrong_test.sort_values(['short_form'])

,short_form,context,long_form,label,sentences,Prediction
883,A,dominante ( ATR-AD ) . Dicha mutación ( c.1766G > < start > A < end > fig . 2A ) supone la sustitución de un aminoácido,adeniner,1,[CLS] adeniner [SEP] dominante ( ATR-AD ) . Dicha mutación ( c.1766G > < start > A < end > fig . 2A ) supone la sustitución de un aminoácido [SEP],0
974,A,la mutación 1691 G-A del factor V y 20210 G- < start > A < end > del factor II . El resto del estudio incluyó hemograma,adenina,1,[CLS] adenina [SEP] la mutación 1691 G-A del factor V y 20210 G- < start > A < end > del factor II . El resto del estudio incluyó hemograma [SEP],0
976,A,realizándose posteriormente una electroforesis que identificó la mutación 1691 G- < start > A < end > del factor V y 20210 G-A del factor II .,adenina,1,[CLS] adenina [SEP] realizándose posteriormente una electroforesis que identificó la mutación 1691 G- < start > A < end > del factor V y 20210 G-A del factor II . [SEP],0
1525,AL,( EVA < 2 ) con la nueva concentración de < start > AL < end > El catéter epidural se retiró en el quinto día del,amiloidosis primario,1,[CLS] amiloidosis primario [SEP] ( EVA < 2 ) con la nueva concentración de < start > AL < end > El catéter epidural se retiró en el quinto día del [SEP],0
1527,AL,horas después de reducir la concentración de anestésico local ( < start > AL < end > a la mitad . No se registraron otros eventos adversos,amiloidosis primario,1,[CLS] amiloidosis primario [SEP] horas después de reducir la concentración de anestésico local ( < start > AL < end > a la mitad . No se registraron otros eventos adversos [SEP],0
...,...,...,...,...,...,...
161,μg,"extraen nuevas concentraciones plasmáticas , siendo aún inferiores ( 8,4 < start > μg < end > mL ) ( se valora la posibilidad de error en",microgramo,1,"[CLS] microgramo [SEP] extraen nuevas concentraciones plasmáticas , siendo aún inferiores ( 8,4 < start > μg < end > mL ) ( se valora la posibilidad de error en [SEP]",0
170,μg,"y elevación de concentraciones plasmáticas predosis de vancomicina hasta 43,1 < start > μg < end > mL , por lo que se suspende tratamiento . Se",picogramo,1,"[CLS] picogramo [SEP] y elevación de concentraciones plasmáticas predosis de vancomicina hasta 43,1 < start > μg < end > mL , por lo que se suspende tratamiento . Se [SEP]",0
411,μg,"días después se inyectan , de forma intravítrea , 25 < start > μg < end > de r-TPA en un volumen de 0,1 ml , seguidos",microgramo,1,"[CLS] microgramo [SEP] días después se inyectan , de forma intravítrea , 25 < start > μg < end > de r-TPA en un volumen de 0,1 ml , seguidos [SEP]",0
1132,μg,enero de 2008 Durogesic® 100 μg/h más fentanilo transmucosa 400 < start > μg < end > de rescate . En una de las prescripciones médicas de,microgramo,1,[CLS] microgramo [SEP] enero de 2008 Durogesic® 100 μg/h más fentanilo transmucosa 400 < start > μg < end > de rescate . En una de las prescripciones médicas de [SEP],0


In [60]:
wrong_test.groupby('short_form').size().reset_index().sort_values(0, ascending = False)

,short_form,0
59,mg,360
4,AV,39
57,c,17
44,PL,13
29,G,12
...,...,...
14,CD4,2
15,CD8,2
36,L,2
30,HB,2


Estudiamos si aquellos acrónimos que no acierta no tenían muchos registros en train

In [76]:
wrong_lf = wrong_test['long_form'].unique().tolist()

In [77]:
group_lf = train.groupby('long_form').size().reset_index().sort_values(0)

In [78]:
group_lf[group_lf['long_form'].isin(wrong_lf)]

,long_form,0
32,coding dno,2
33,coding dno sequence,2
49,enzymelinked immunosorbent assay,3
44,cumulo diferenciacion 4,3
0,adenina,3
1,adeniner,3
8,analisis inmunoabsorcion ligado enzima,3
110,tercero vertebra cervical,4
109,tercero componente complemento,4
45,cumulo diferenciacion 8,4


Estudiemos cada acrónimo que da fallo

In [62]:
wrong_test[wrong_test['short_form'] == 'mg'][['short_form', 'long_form', 'label', 'sentences', 'Prediction']]

,short_form,long_form,label,sentences,Prediction
5,mg,miligramo,1,"[CLS] miligramo [SEP] . La analítica mostró los siguientes datos : bilirrubina 19,44 < start > mg < end > dl , GOT 110 UI/ml , GPT 371 UI/ml , [SEP]",0
7,mg,miligramo,1,[CLS] miligramo [SEP] inició tratamiento con corticoides ( metilprednisolona a dosis de 32 < start > mg < end > día ) y se retiró el drenaje percutáneo . Con [SEP],0
12,mg,miligramo,1,[CLS] miligramo [SEP] ( 600 mg/6 horas ) y Tobramicina ( 50 m < start > g/ < end > 12 horas ) . Las muestras obtenidas para cultivo dieron [SEP],0
14,mg,miligramo,1,"[CLS] miligramo [SEP] ( 400 mg/12 horas ) , Clindamicina ( 600 m < start > g/ < end > horas ) y Tobramicina ( 50 mg/ 12 horas ) [SEP]",0
16,mg,miligramo,1,"[CLS] miligramo [SEP] ( 400 mg/12 horas ) , Teicoplanina ( 400 m < start > g/ < end > 2 horas ) , Clindamicina ( 600 mg/6 horas ) [SEP]",0
...,...,...,...,...,...
1665,mg,miligramo,1,"[CLS] miligramo [SEP] mantiene tratamiento farmacológico pautado desde CSMIJ ( Sertralina , 100 < start > mg < end > , y se deriva a psicología para trabajar dificultades sociales [SEP]",0
1673,mg,miligramo,1,"[CLS] miligramo [SEP] bioquímica muestra un mal control glucémico ( glucosa : 571 < start > mg < end > dL ) , datos de colestasis y citolisis ( bilirrubina [SEP]",0
1675,mg,miligramo,1,"[CLS] miligramo [SEP] ) y una insuficiencia renal aguda ( creatinina : 2,8 < start > mg < end > dL ) . El hemograma es prácticamente normal ( leucocitos [SEP]",0
1677,mg,miligramo,1,"[CLS] miligramo [SEP] bilirrubina total : 10,9 mg/dL ; bilirrubina directa : 9,3 < start > mg < end > dL ; GOT : 57 U/L ; GPT : 134 [SEP]",0


In [63]:
test[(test['short_form'] == 'mg')]['long_form'].unique()

array(['miligramo', 'miligramar'], dtype=object)

In [67]:
test[(test['short_form'] == 'mg') & (test['long_form'] == 'miligramar') & (test['label'] == 1)]

,short_form,context,long_form,label,sentences,Prediction
748,mg,"su centro de atención primaria presenta una calcemia de 15,35 < start > mg < end > dL ( valores de referencia [ v.r . ] entre",miligramar,1,"[CLS] miligramar [SEP] su centro de atención primaria presenta una calcemia de 15,35 < start > mg < end > dL ( valores de referencia [ v.r . ] entre [SEP]",0
750,mg,"6,3 mg/dL ( v.r . entre 4,2 mg/dL y 5,4 < start > mg < end > dL ) , con una exploración física , radiografía de",miligramar,1,"[CLS] miligramar [SEP] 6,3 mg/dL ( v.r . entre 4,2 mg/dL y 5,4 < start > mg < end > dL ) , con una exploración física , radiografía de [SEP]",0
752,mg,"calcio iónico de 6,3 mg/dL ( v.r . entre 4,2 < start > mg < end > dL y 5,4 mg/dL ) , con una exploración física",miligramar,1,"[CLS] miligramar [SEP] calcio iónico de 6,3 mg/dL ( v.r . entre 4,2 < start > mg < end > dL y 5,4 mg/dL ) , con una exploración física [SEP]",0
754,mg,"calcemia de 16,1 mg/dL con un calcio iónico de 6,3 < start > mg < end > dL ( v.r . entre 4,2 mg/dL y 5,4 mg/dL",miligramar,1,"[CLS] miligramar [SEP] calcemia de 16,1 mg/dL con un calcio iónico de 6,3 < start > mg < end > dL ( v.r . entre 4,2 mg/dL y 5,4 mg/dL [SEP]",0
756,mg,"se realiza una analítica que muestra una calcemia de 16,1 < start > mg < end > dL con un calcio iónico de 6,3 mg/dL ( v.r",miligramar,1,"[CLS] miligramar [SEP] se realiza una analítica que muestra una calcemia de 16,1 < start > mg < end > dL con un calcio iónico de 6,3 mg/dL ( v.r [SEP]",0
758,mg,"referencia [ v.r . ] entre 8,8 mg/dL y 10,2 < start > mg < end > dL ) . En urgencias se realiza una analítica que",miligramar,1,"[CLS] miligramar [SEP] referencia [ v.r . ] entre 8,8 mg/dL y 10,2 < start > mg < end > dL ) . En urgencias se realiza una analítica que [SEP]",0
760,mg,"( valores de referencia [ v.r . ] entre 8,8 < start > mg < end > dL y 10,2 mg/dL ) . En urgencias se realiza",miligramar,1,"[CLS] miligramar [SEP] ( valores de referencia [ v.r . ] entre 8,8 < start > mg < end > dL y 10,2 mg/dL ) . En urgencias se realiza [SEP]",0


In [64]:
test[(test['short_form'] == 'mg') & (test['label'] == 1)]

,short_form,context,long_form,label,sentences,Prediction
5,mg,". La analítica mostró los siguientes datos : bilirrubina 19,44 < start > mg < end > dl , GOT 110 UI/ml , GPT 371 UI/ml ,",miligramo,1,"[CLS] miligramo [SEP] . La analítica mostró los siguientes datos : bilirrubina 19,44 < start > mg < end > dl , GOT 110 UI/ml , GPT 371 UI/ml , [SEP]",0
7,mg,inició tratamiento con corticoides ( metilprednisolona a dosis de 32 < start > mg < end > día ) y se retiró el drenaje percutáneo . Con,miligramo,1,[CLS] miligramo [SEP] inició tratamiento con corticoides ( metilprednisolona a dosis de 32 < start > mg < end > día ) y se retiró el drenaje percutáneo . Con [SEP],0
12,mg,( 600 mg/6 horas ) y Tobramicina ( 50 m < start > g/ < end > 12 horas ) . Las muestras obtenidas para cultivo dieron,miligramo,1,[CLS] miligramo [SEP] ( 600 mg/6 horas ) y Tobramicina ( 50 m < start > g/ < end > 12 horas ) . Las muestras obtenidas para cultivo dieron [SEP],0
14,mg,"( 400 mg/12 horas ) , Clindamicina ( 600 m < start > g/ < end > horas ) y Tobramicina ( 50 mg/ 12 horas )",miligramo,1,"[CLS] miligramo [SEP] ( 400 mg/12 horas ) , Clindamicina ( 600 m < start > g/ < end > horas ) y Tobramicina ( 50 mg/ 12 horas ) [SEP]",0
16,mg,"( 400 mg/12 horas ) , Teicoplanina ( 400 m < start > g/ < end > 2 horas ) , Clindamicina ( 600 mg/6 horas )",miligramo,1,"[CLS] miligramo [SEP] ( 400 mg/12 horas ) , Teicoplanina ( 400 m < start > g/ < end > 2 horas ) , Clindamicina ( 600 mg/6 horas ) [SEP]",0
...,...,...,...,...,...,...
1665,mg,"mantiene tratamiento farmacológico pautado desde CSMIJ ( Sertralina , 100 < start > mg < end > , y se deriva a psicología para trabajar dificultades sociales",miligramo,1,"[CLS] miligramo [SEP] mantiene tratamiento farmacológico pautado desde CSMIJ ( Sertralina , 100 < start > mg < end > , y se deriva a psicología para trabajar dificultades sociales [SEP]",0
1673,mg,"bioquímica muestra un mal control glucémico ( glucosa : 571 < start > mg < end > dL ) , datos de colestasis y citolisis ( bilirrubina",miligramo,1,"[CLS] miligramo [SEP] bioquímica muestra un mal control glucémico ( glucosa : 571 < start > mg < end > dL ) , datos de colestasis y citolisis ( bilirrubina [SEP]",0
1675,mg,") y una insuficiencia renal aguda ( creatinina : 2,8 < start > mg < end > dL ) . El hemograma es prácticamente normal ( leucocitos",miligramo,1,"[CLS] miligramo [SEP] ) y una insuficiencia renal aguda ( creatinina : 2,8 < start > mg < end > dL ) . El hemograma es prácticamente normal ( leucocitos [SEP]",0
1677,mg,"bilirrubina total : 10,9 mg/dL ; bilirrubina directa : 9,3 < start > mg < end > dL ; GOT : 57 U/L ; GPT : 134",miligramo,1,"[CLS] miligramo [SEP] bilirrubina total : 10,9 mg/dL ; bilirrubina directa : 9,3 < start > mg < end > dL ; GOT : 57 U/L ; GPT : 134 [SEP]",0


In [30]:
acr = 'PCR'

text = 'Nuria y Poveda se han hecho una PCR por la fiesta del sábado'

In [31]:
start_i = text.find(acr)
end_i = start_i + len(acr)

In [32]:
text[start_i:end_i]

'PCR'